In [1]:
import numpy as np
import sys
import os
from tqdm import trange
sys.path.append('../python') #Path to load WI_Solver_Utils.py
import WI_Solver_utils
from WI_Solver_utils import InflatonModel, Background, Perturbations

In [15]:

##############################
## Define Global Parameters: #
##############################

Mpl = 1 # Everything is in Plank units M_{pl}
g = 228.27  # SUSY relativistic degrees of freedom
a1 = np.pi**2/30*g

Ne_inflation=60 #Number of e-folds of after CMB horizon crossing
Ne_pert= 10 # Number of e-folds before CMB horizon crossing

#\tau Array:#

tau_ini = 6
tau_end = -1
N = 100000
# dtau is a negative quantity as expected since tau is decreasing over time
dtau = (tau_end - tau_ini) / N
taus = np.linspace(tau_ini, tau_end, N)


#Q Array:#

Qs=[9.30000000e+03, 8.00000000e+03, 7.94328235e+03, 6.47308204e+03,5.27499706e+03, 5.00000000e+03, 4.29866235e+03, 3.98107000e+03, 3.50303474e+03, 2.85466766e+03, 2.32630507e+03, 1.89573565e+03, 1.58489000e+03, 1.54485915e+03, 1.25892541e+03, 1.00000000e+03, 6.60693448e+02, 6.30957000e+02, 4.36515832e+02, 2.88403150e+02, 2.51189000e+02, 1.90546072e+02, 1.25892541e+02, 1.00000000e+02, 8.60000000e+01, 7.60000000e+01, 6.60000000e+01, 5.60000000e+01, 4.60000000e+01, 3.60000000e+01, 2.60000000e+01, 1.60000000e+01,6.00000000e+00, 3.16227766e+00, 1.00000000e+00, 4.86967525e-01, 2.37137000e-01, 7.49894209e-02, 5.62341000e-02, 1.33352000e-02, 1.15478198e-02, 3.16228000e-03, 1.77827941e-03, 7.49894000e-04, 2.73841963e-04, 1.77828000e-04, 4.21697000e-05, 4.21696503e-05, 1.00000000e-05, 6.49381632e-06, 1.00000000e-06, 1.00000000e-07]


############################################################################################################
###                             EXAMPLE 1: MONOMIAL QUARTIC POTENTIAL                                    ###
############################################################################################################


lv = 10**(-14)  # value of lambda

Model = InflatonModel('monomial', [lv*np.math.factorial(4), 4], g, a1, Mpl)

ph0_ini=10
lr=.01 #Initializing the value of the learning rate
ICs=[]
for i in trange(len(Qs)):
    #Call the background, initializing ph0 at any point which we call ph0_ini
    Bg = Background(Model,ph0_ini , Qs[i])
    # Us the slow-roll guesser in the Background module to guess a reasonable value for ph0:
    ph0_guess=Bg.Phi0_slow_roll_guesser(Ne_inflation+Ne_pert)
    
    '''Reinitiate the background module with ph0_guess and then start the iterative background solver: Note always better to start from a very small value for the learning_rate
    and then increase it if the corvengence is too slow'''
    Bg = Background(Model,ph0_guess , Qs[i])
    if (i>=15) and(i<30):
        lr=1
    if i>=30:
        lr=10
    ph0_val = Bg.Bg_solver_test(Ne_inflation+Ne_pert, int(2*N), Ne_inflation, verbose=False, learning_rate=lr)
    ICs.append([Qs[i],ph0_val])
    ph0_ini=ph0_guess
    del ph0_val, ph0_guess

#Save the new initial conditions in a txt file:
np.savetxt('../ICS_ph0-Q0s/ICS_ph0-Q0s_quartic_Ne'+str(int(Ne_inflation))+'.txt', ICs)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 52/52 [03:24<00:00,  3.94s/it]
